In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os

In [2]:
sc = SparkContext()

22/05/06 17:55:57 WARN Utils: Your hostname, Nathans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
22/05/06 17:55:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/06 17:55:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Requirements:

### Solution

#### Q1

In [7]:
data_file_path = os.path.join('..', 'src-data', 'kddcup.data_10_percent_corrected')
RDD = sc.textFile(data_file_path)

#### Q2

In [8]:
print(f"Number of element: {RDD.count()}")

Number of element: 494021


#### Q3

In [10]:
RDD.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

#### Q4

In [12]:
RDD_normal = RDD.filter(lambda x: 'normal.' in x)

#### Q5

In [13]:
import time

In [14]:
start = time.time()
count_normal = RDD_normal.count()
duration = time.time() - start
print(f"Number of element in RDD_normal: {count_normal}")
print(f"Duration: {duration}")

Number of element in RDD_normal: 97278
Duration: 0.19905519485473633


#### Q6

In [15]:
start = time.time()
lst = RDD.collect()
duration = time.time() - start
print(f"Duration: {duration}")

Duration: 0.7444717884063721


#### Q7
Use `sample()` method.

In [17]:
RDD_sample = RDD.sample(withReplacement = False, fraction = 0.1, seed = 42)
print(f"Sample size:  {RDD_sample.count()} and Original size: {RDD.count()}")

Sample size:  49214 and Original size: 494021


#### Q8

In [18]:
RDD_sample_normal = RDD_sample.filter(lambda x: 'normal.' in x)
start = time.time() 
count_sample_normal = RDD_sample_normal.count()
duration = time.time() - start

print(f"Duration: {duration}")
print(f"RDD sample normal size: {RDD_sample_normal.count()}")
print(f"Fraction: {RDD_sample_normal.count() / RDD_sample.count()}")

Duration: 0.16452407836914062
RDD sample normal size: 9791
Fraction: 0.19894745397651076


#### Q9
Use `subtract()` method.

In [19]:
RDD_sample_without_normal = RDD_sample.subtract(RDD_sample_normal)
print(f"RDD sample without normal size: {RDD_sample_without_normal.count()}")

RDD sample without normal size: 39423


In [20]:
RDD_sample_without_normal.take(1)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,509,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.']

#### Q10

In [23]:
RDD_lst = RDD.map(lambda x: x.split(','))
RDD_lst.first()

['0',
 'tcp',
 'http',
 'SF',
 '181',
 '5450',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8',
 '8',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '1.00',
 '0.00',
 '0.00',
 '9',
 '9',
 '1.00',
 '0.00',
 '0.11',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 '0.00',
 'normal.']

In [24]:
RDD_lst.take(1)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.']]

You can see the different between `first()` and `take(1)` method.

#### Q11
The 'protocal' is second column --> index = 1.
For get unique 'protocal' value --> use `distinct()` method.

In [27]:
# get the second column
RDD_lst.map(lambda x: x[1]).take(5)

['tcp', 'tcp', 'tcp', 'tcp', 'tcp']

In [31]:
RDD_lst.map(lambda x: x[1]).distinct()

PythonRDD[46] at RDD at PythonRDD.scala:53

In [29]:
type(RDD_lst.map(lambda x: x[1]).distinct())

pyspark.rdd.PipelinedRDD

In [34]:
unique_protocals = RDD_lst.map(lambda x: x[1]).distinct()
print(f"unique protocals: {unique_protocals.collect()}")
print(f"number of unique protocals: {unique_protocals.count()}")

unique protocals: ['tcp', 'udp', 'icmp']
number of unique protocals: 3


#### Q12

In [35]:
unique_services = RDD_lst.map(lambda x: x[2]).distinct()
print(f"unique services: {unique_services.collect()}")
print(f"number of unique services: {unique_services.count()}")

unique services: ['http', 'domain_u', 'auth', 'pop_3', 'rje', 'whois', 'login', 'imap4', 'supdup', 'csnet_ns', 'sunrpc', 'uucp_path', 'netbios_dgm', 'Z39_50', 'netstat', 'urh_i', 'urp_i', 'pm_dump', 'tftp_u', 'red_i', 'smtp', 'finger', 'telnet', 'eco_i', 'ecr_i', 'private', 'ftp_data', 'mtp', 'link', 'gopher', 'ssh', 'name', 'ctf', 'shell', 'IRC', 'efs', 'courier', 'echo', 'iso_tsap', 'hostnames', 'netbios_ns', 'vmnet', 'bgp', 'ftp', 'ntp_u', 'other', 'time', 'remote_job', 'domain', 'daytime', 'nntp', 'nnsp', 'http_443', 'exec', 'printer', 'uucp', 'klogin', 'kshell', 'discard', 'systat', 'pop_2', 'netbios_ssn', 'sql_net', 'ldap', 'X11', 'tim_i']
number of unique services: 66


#### Q13
Use `cartesian` method. [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.cartesian.html)

In [37]:
type(unique_protocals.cartesian(unique_services))

pyspark.rdd.RDD

In [40]:
products = unique_protocals.cartesian(unique_services)
products.collect()[:10] # print first 10 rows.

[('tcp', 'http'),
 ('tcp', 'domain_u'),
 ('tcp', 'auth'),
 ('tcp', 'pop_3'),
 ('tcp', 'rje'),
 ('tcp', 'whois'),
 ('tcp', 'login'),
 ('tcp', 'imap4'),
 ('tcp', 'supdup'),
 ('tcp', 'csnet_ns')]

In [41]:
print(f"Number of combination of protocal x services: {products.count()}")

Number of combination of protocal x services: 198


#### Q14

In [42]:
RDD_lst.getNumPartitions()

3

#### Q15

In [44]:
save_RDD_file_path = os.path.join('..', 'src-data', 'kdd_cup')
RDD_lst.saveAsTextFile(save_RDD_file_path)